# Example of solvus construction using convex hulls
## Three component example

In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.interpolate import interp1d, interp2d
from thermoengine import model, phases
import matplotlib.pyplot as plt
%matplotlib inline

Use feldspar as a test case ...

In [ ]:
modelDB = model.Database()
Feldspar = modelDB.get_phase('Fsp')

In [ ]:
Feldspar.props['endmember_name']

## Construct Gibbs energy of solution ...

In [ ]:
t = 1000
p = 1000
X = []
g = []
muAb = []
muAn = []
muSn = []
mu0Ab = Feldspar.gibbs_energy(t, p, mol=np.array([1.0,0.0,0.0]))
mu0An = Feldspar.gibbs_energy(t, p, mol=np.array([0.0,1.0,0.0]))
mu0Sn = Feldspar.gibbs_energy(t, p, mol=np.array([0.0,0.0,1.0]))
print (mu0Ab, mu0An, mu0Sn)
for i in range(0,101):
    for j in range(0,101):
        if i+j <= 100:
            X.append([i*0.01, j*0.01, (100.-i-j)/100.])
            ss = i*0.01*mu0Ab + j*0.01*mu0An + (100.-i-j)*mu0Sn/100.
            gmix = Feldspar.gibbs_energy(t, p, mol=np.array([i*0.01, j*0.01, (100.-i-j)/100.])) - ss
            #print(i, j, 100-i-j, ss, gmix)
            g.append(gmix)
            mu = Feldspar.chem_potential(t,p,mol=np.array([i*0.01, j*0.01, (100.-i-j)/100.]))[0]
            muAb.append(mu[0])
            muAn.append(mu[1])
            muSn.append(mu[2])
X = np.array(X)
g = np.array(g)
muAb = np.array(muAb)
muAn = np.array(muAn)
muSn = np.array(muSn)

## Construct a ternary contour plot ...

In [ ]:
import plotly.figure_factory as ff

In [ ]:
fig = ff.create_ternary_contour(np.array([X[:,0], X[:,1], X[:,2]]), g, 
                                pole_labels=['Ab', 'An', 'Sn'], interp_mode='cartesian',
                                ncontours=20,
                                colorscale='Viridis',
                                showscale=True,
                                title='Gibbs free energy of feldspar')
fig.show(renderer='iframe')

In [ ]:
import ternary
print("Version", ternary.__version__)
import matplotlib
matplotlib.rcParams['figure.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (4, 4)

In [ ]:
import math
def Gibbs_free_energy(point):
    result = Feldspar.gibbs_energy(t, p, mol=np.array(point))
    result -= point[0]*mu0Ab + point[1]*mu0An + point[2]*mu0Sn
    return result

scale = 100

figure, tax = ternary.figure(scale=scale)
figure.set_size_inches(10, 8)
tax.heatmapf(Gibbs_free_energy, boundary=True, style="triangular")
tax.boundary(linewidth=2.0)
tax.set_title("Gibbs free energy")
tax.ticks(axis='lbr', linewidth=1, multiple=5)
tax.clear_matplotlib_ticks()
tax.show()

## Plot a Gibbs free energy pseudosection ...
 $y/z = r$  
$x+y+z=1$  
$x + zr + z = 1$  
$z (r+1) = 1 - x$  
$z = (1-x)/(r+1)$  
$y = r (1-x)/(1+r)$

In [ ]:
ratio = 1
xx = []
yy = []
for i in range(0,101):
    tAb = i*0.01
    tAn = ratio*(1.-tAb)/(1.+ratio)
    tSn = (1.-tAb)/(1.+ratio)
    xx.append(tAb)
    ss = tAb*mu0Ab + tAn*mu0An + tSn*mu0Sn
    yy.append(Feldspar.gibbs_energy(t, p, mol=np.array([tAb, tAn, tSn]))-ss)
xx = np.array(xx)
yy = np.array(yy)
fig, ax = plt.subplots()
ax.plot(xx, yy, 'r-')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for j in range(1,2):
    xx = []
    yy = []
    tAb = 0.1*j
    for i in range(0,201):
        tAn = i*0.005
        tSn = 1.0 - tAn - tAb
        if tSn > 0 and tAb+tAn+tSn <= 1.0:
            xx.append(tAn)
            ss = tAb*mu0Ab + tAn*mu0An + tSn*mu0Sn
            ss = 0
            yy.append(Feldspar.gibbs_energy(t, p, mol=np.array([tAb, tAn, tSn]))-ss)
    xx = np.array(xx)
    yy = np.array(yy)
    ax.plot(xx, yy, 'r-')
plt.show()

## Construct a convex hull of the Gibbs function ...
and an interpolating function

## Plot the Gibbs function and the convex hull ...

In [ ]:
fig, ax = plt.subplots()
ax.plot(xx, yy, 'k-')
ax.plot(x_hull, y_hull, 'ro')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax1 = plt.subplot(121)
ax1.plot(np.indices(x_hull.shape)[0], x_hull, 'ro')
ax2 = plt.subplot(122)
ax2.plot(np.indices(x_hull.shape)[0], y_hull, 'bo')
plt.show()

## Extract the titeline and plot ...

In [ ]:
temp = np.nonzero(g-f_hull(mole_frac))[0]
tieline = []
if np.size(temp) > 0:
    if temp[0] > 0:
        tieline.append((mole_frac[temp[0]-1]+mole_frac[temp[0]])/2)
    else:
        tieline.append(mole_frac[temp[0]])
    if temp[-1] < 1:
        tieline.append((mole_frac[temp[-1]]+mole_frac[temp[-1]+1])/2)
    else:
        tieline.append(mole_frac[temp[-1]])
print ('Solvus tieline at:', tieline)

fig, ax = plt.subplots()
ax.plot(mole_frac, g, 'k-')
ax.plot(tieline, f_hull(tieline), 'r-')
plt.show()